In [246]:
# 모듈 import
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

In [247]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
#cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
#cfg=Config.fromfile('./configs/yolof/yolof_r50_c5_8x8_1x_coco.py')
cfg=Config.fromfile('./configs/yolof/yolof_r50_c5_8x8_iter-1x_coco.py')
root='../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
#stratified kfold
cfg.data.train.ann_file=root+'stratified_v2/train_fold_1.json'
#cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize



cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
#stratified k fold
cfg.data.val.classes = classes
cfg.data.val.img_prefix=root
cfg.data.val.ann_file=root+'stratified_v2/val_fold_1.json'
print(cfg.data.val.pipeline)
cfg.data.val.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
#cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'
cfg.work_dir='./work_dirs/retinanet_effb3_fpn_crop896_8x4_1x_coco'

#cfg.model.roi_heads.bbox_head.num_classes = 10
cfg.model.bbox_head.num_classes = 10

#yolo 전용 파라미터 수정
cfg.model.test_cfg.score_thr=0.1
#이건 직접가서 수정함 100에서 30으로 바꿈
#cfg.model.test_cfg.nms['max_per_img']=30
#학습률도 바꿧음
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

[{'type': 'LoadImageFromFile'}, {'type': 'MultiScaleFlipAug', 'img_scale': (512, 512), 'flip': False, 'transforms': [{'type': 'Resize', 'keep_ratio': True}, {'type': 'RandomFlip'}, {'type': 'Normalize', 'mean': [103.53, 116.28, 123.675], 'std': [1.0, 1.0, 1.0], 'to_rgb': False}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'ImageToTensor', 'keys': ['img']}, {'type': 'Collect', 'keys': ['img']}]}]


In [248]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]
#valid까지 확인
#datasets=[build_dataset(cfg.data.train),build_dataset(cfg.data.val)]

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [249]:
# dataset 확인
datasets[0]



CocoDataset Train dataset with number of images 3914, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3161  | 1 [Paper]     | 5115  | 2 [Paper pack]  | 706   | 3 [Metal]   | 769   | 4 [Glass]    | 835   |
| 5 [Plastic]       | 2350  | 6 [Styrofoam] | 1026  | 7 [Plastic bag] | 4151  | 8 [Battery] | 143   | 9 [Clothing] | 377   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [250]:
# 모델 build 및 pretrained network 불러오기

model = build_detector(cfg.model)
#model.init_weights()

In [251]:
# #wandb테스트 여기서 이렇게 코드를 쳐서 실행하나 ./configs/_base_/default_runtime.py에서 수정하나 똑같은듯?
# import wandb
# wandb.init(project="level2-objectdetection", entity="hi-ai")
# cfg.log_config.hooks=[
#     dict(type='TextLoggerHook'),
#     dict(type='MMDetWandbHook',
#          init_kwargs={'project':'level2-objectdetection'},
#          interval=10,
#          log_checkpoint=True,
#          log_checkpoint_metadata=True,
#          num_eval_images=100)]

import wandb
wandb.init(project="Model", entity="hi-ai")

log_config = dict(
    interval=10,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(
            type='WandbLoggerHook',
            init_kwargs=dict(project='Model', name='yolof', entity='hi-ai')
            )])
# for wandb
cfg.log_config = log_config

# import wandb
# wandb.init(project="level2-objectdetection", entity="hi-ai")


# 모델 학습

train_detector(model, datasets, cfg, distributed=False, validate=True)

# preds = []
# gts = []

# # 테스트 데이터의 예측 결과를 추출합니다.
# for i, data in enumerate(data_loader):
#     with torch.no_grad():
#         result = single_gpu_test(model, data["img"][0].cuda(), show=False)
#     preds.extend(result[0])
#     gts.extend(data["gt_labels"][0].tolist())

# # confusion matrix와 accuracy를 계산합니다.
# cm = eval_map(preds, gts, iou_thr=0.5, print_summary=False, dataset=dataset.CLASSES)
# acc = sum([1 for i, j in zip(preds, gts) if i == j]) / len(preds)

# wandb에 confusion matrix와 accuracy를 로깅합니다.
# wandb.log({"confusion_matrix": wandb.plot.confusion_matrix(preds, gts, class_names=dataset.CLASSES),
#            "accuracy": acc})

learning_rate,▁▁▂▃▄▅▅▆▇███████████████████████████████
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/grad_norm,█▇▄▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▇▇▄▃▇▄▃▃▃▃▂▃▂▂▂▂▂▃▂▃▂▁▁▂▂▂▂▃▃▁▂▁▂▁▂▁▂▃▁
train/loss_bbox,▅▃▂▄▃█▅▄▃▄▄▂▃▂▃▃▃▂▃▂▃▂▂▂▁▂▄▃▃▄▂▂▁▂▂▃▁▃▄▁
train/loss_cls,▇▆█▃▂▄▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▁▁▁▂▂▁▁▂▂▂▂
val/bbox_mAP,▁▆▁▆██
val/bbox_mAP_50,▁▃▂▆██
val/bbox_mAP_75,▁▁▁▁▁▁
val/bbox_mAP_l,▁▄▁███
val/bbox_mAP_m,▁▁▁▁▁▁


2023-05-11 04:04:04,450 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2023-05-11 04:04:04,545 - mmdet - INFO - Start running, host: root@718ec73291f9, work_dir: /opt/ml/baseline/mmdetection/work_dirs/retinanet_effb3_fpn_crop896_8x4_1x_coco
2023-05-11 04:04:04,546 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
bef

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


2023-05-11 04:04:16,296 - mmdet - INFO - Saving checkpoint at 1 iterations
2023-05-11 04:04:17,748 - mmdet - INFO - Saving checkpoint at 2 iterations
2023-05-11 04:04:18,976 - mmdet - INFO - Saving checkpoint at 3 iterations
2023-05-11 04:04:20,077 - mmdet - INFO - Saving checkpoint at 4 iterations
2023-05-11 04:04:21,255 - mmdet - INFO - Saving checkpoint at 5 iterations
2023-05-11 04:04:22,438 - mmdet - INFO - Saving checkpoint at 6 iterations
2023-05-11 04:04:23,577 - mmdet - INFO - Saving checkpoint at 7 iterations
2023-05-11 04:04:24,726 - mmdet - INFO - Saving checkpoint at 8 iterations
2023-05-11 04:04:25,905 - mmdet - INFO - Saving checkpoint at 9 iterations
2023-05-11 04:04:27,088 - mmdet - INFO - Saving checkpoint at 10 iterations
2023-05-11 04:04:28,134 - mmdet - INFO - Iter [10/22500]	lr: 7.995e-04, eta: 7:39:11, time: 1.225, data_time: 0.029, memory: 4215, loss_cls: 4.1752, loss_bbox: 0.8443, loss: 5.0195, grad_norm: 88.1426
2023-05-11 04:04:28,271 - mmdet - INFO - Saving 

[>>>>>>>>>>>>>>>>>>>          ] 639/969, 37.3 task/s, elapsed: 17s, ETA:     9s

...

Loading and preparing results...
DONE (t=0.04s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.60s).
Accumulating evaluation results...


2023-05-11 08:49:36,101 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.009
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.032
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.001
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.011
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.071
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.071
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.071
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.001
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=1.15s).


2023-05-11 08:49:36,345 - mmdet - INFO - Saving checkpoint at 13501 iterations
2023-05-11 08:49:37,833 - mmdet - INFO - Saving checkpoint at 13502 iterations
2023-05-11 08:49:39,222 - mmdet - INFO - Saving checkpoint at 13503 iterations
2023-05-11 08:49:40,550 - mmdet - INFO - Saving checkpoint at 13504 iterations
2023-05-11 08:49:41,795 - mmdet - INFO - Saving checkpoint at 13505 iterations
2023-05-11 08:49:43,046 - mmdet - INFO - Saving checkpoint at 13506 iterations
2023-05-11 08:49:44,297 - mmdet - INFO - Saving checkpoint at 13507 iterations
2023-05-11 08:49:45,652 - mmdet - INFO - Saving checkpoint at 13508 iterations
2023-05-11 08:49:47,049 - mmdet - INFO - Saving checkpoint at 13509 iterations
2023-05-11 08:49:48,335 - mmdet - INFO - Saving checkpoint at 13510 iterations
2023-05-11 08:49:49,476 - mmdet - INFO - Iter [13510/22500]	lr: 1.200e-01, eta: 3:10:01, time: 4.705, data_time: 3.389, memory: 4215, loss_cls: 0.5305, loss_bbox: 0.6510, loss: 1.1815, grad_norm: 0.5699
2023-05

OSError: [Errno 28] No space left on device